In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import os
import glob

from ssd_training import plot_log

In [ ]:
experiments = !ls -1 ./checkpoints
print(experiments.n)

### Log

In [ ]:
checkdir = './checkpoints/' + experiments[-1]
#checkdir = './checkpoints/201710132146_tb300_synthtext_horizontal10'
#checkdir = './checkpoints/201710141431_tb300_synthtext_horizontal10'
#checkdir = './checkpoints/201710041551_dsod300_voc'

names = [
    'loss',
    #'lr', 
    'pos_conf_loss', 'pos_loc_loss', 'neg_conf_loss',
    # SegLink
    'seg_conf_loss', 'seg_loc_loss', 'link_conf_loss',
    #'pos_seg_conf_loss', 'neg_seg_conf_loss', 'pos_link_conf_loss', 'neg_link_conf_loss', 
    #'num_pos_seg', 'num_neg_seg', 
    #'num_pos_link', 'num_neg_link',
    'seg_fmeasure', 'link_fmeasure',
    #'seg_precision', 'seg_recall',
    #'link_precision', 'link_recall',
    # TBPP
    'conf_loss', 'loc_loss',
    'precision', 'recall', 
    'fmeasure'
]
#names = None

limits = [100, np.inf]
#limits = None
#limits = [0, 10000]

plot_log(checkdir+'/log.json', names, limits, window_length=4000)

### History

In [ ]:
with open(checkdir+'/history.json','r') as f:
    hist = json.load(f)
#print(list(hist.keys()))
epochs = np.arange(1,len(hist['loss'])+1)
plt.figure(figsize=(12,6))
plt.plot(epochs, hist['loss'])
plt.plot(epochs, hist['val_loss'])
plt.title('loss')
plt.legend(['training','validation'])
ax = plt.gca()
ticks = np.arange(len(hist['loss']))+1
ax.set_xticks(ticks)
ax.set_xticklabels(ticks)
plt.grid()
#plot_name = 'dsodsl512_sythtext_b'
#plt.savefig('plots/%s_loss.pgf' % (plot_name), bbox_inches='tight')
#plt.ylim([0.0, 5.0])
plt.show()

In [ ]:
signals = [
    # SSD
    #'pos_conf_loss', 'neg_conf_loss', 'pos_loc_loss',
    #'precision', 'recall', 'fmeasure',
    # SegLink
    #'seg_precision', 'seg_recall', 'seg_fmeasure',
    #'link_precision', 'link_recall', 'link_fmeasure',
    #'pos_seg_conf_loss', 'neg_seg_conf_loss', 'seg_loc_loss', 'pos_link_conf_loss', 'neg_link_conf_loss',
    #'num_pos_seg', 'num_neg_seg', 'num_pos_link', 'num_neg_link', 
    'seg_conf_loss', 'seg_loc_loss', 'link_conf_loss', 
]

fig, axs = plt.subplots(1, len(signals), figsize=(20,4))
for i, s in enumerate(signals):
    if s not in hist.keys():
        print('missing %s' %(s))
        continue
    axs[i].plot(epochs, hist[s])
    axs[i].plot(epochs, hist['val_'+s])
    axs[i].set_title(s)
    if s.split('_')[-1] in ['precision', 'recall', 'fmeasure']:
        axs[i].set_ylim([0,1])
    ticks = np.arange(len(hist['loss']))+1
    axs[i].set_xticks(ticks)
    axs[i].set_xticklabels(ticks)
    axs[i].grid()
#plt.savefig('plots/%s_links.pgf' % (plot_name), bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=[12,6])
x = np.arange(len(hist['val_loss']))+1
n = 100
plt.plot(x[-n:], hist['val_loss'][-n:])
plt.grid()
plt.show()

### Compare Log

In [ ]:
# DSOD recursive

checkdir1 = './checkpoints/201808261035_dsod300fl_voccoco'
#checkdir1 = './checkpoints/201809012037_ssd512_recursive_voccoco'
#checkdir1 = './checkpoints/201809031002_ssd512_recursive_voccoco'
#checkdir2 = './checkpoints/201809032250_ssd512_recursive_voccoco'
checkdir2 = './checkpoints/201809041729_ssd512_recursive_voccoco'
names = [
    'loss',
    'conf_loss', 'loc_loss',
    'precision', 'recall', 
    'fmeasure',
]

limits = [0, 100000]
limits = None

plot_log(checkdir1+'/log.json', names, limits, window_length=2000, log_file_compare=checkdir2+'/log.json')

In [ ]:
# SegLink

#checkdir1 = './checkpoints/201711132011_dsodsl512_synthtext' # 2 epochs, OHNM
#checkdir2 = './checkpoints/201805161634_dsodsl512_synthtext' # 2 epochs, Focal Loss
#checkdir1 = './checkpoints/201806021007_dsodsl512_synthtext' # 24 eopchs, 1/4 of training set
checkdir1 = './checkpoints/201711071436_sl512_synthtext'
checkdir2 = './checkpoints/201809231008_sl512_synthtext'

names = [
    'loss',
    'pos_conf_loss', 'pos_loc_loss', 'neg_conf_loss',
    'seg_conf_loss', 'pos_seg_conf_loss', 'neg_seg_conf_loss',
    'seg_loc_loss',
    'link_conf_loss', 'pos_link_conf_loss', 'neg_link_conf_loss',
    
    'seg_fmeasure', 'link_fmeasure',
    'seg_precision', 'seg_recall',
    'link_precision', 'link_recall',
]

plot_log(checkdir1+'/log.json', names, [10, np.inf], window_length=2000, log_file_compare=checkdir2+'/log.json')

In [ ]:
# TextBoxes++

checkdirs = !ls -1d ./checkpoints/*tbpp512fl_*
print(checkdirs.n, '\n')

checkdir1, checkdir2 = checkdirs[0], checkdirs[3]
checkdir1, checkdir2 = checkdirs[3], checkdirs[4]

plot_log(checkdir1+'/log.json', ['loss', 'conf_loss', 'loc_loss', 'precision', 'recall', 'fmeasure'], [0*10000,np.inf], 
         log_file_compare=checkdir2+'/log.json',
         window_length=2500)

### Log playground

In [ ]:
limits = None

with open(checkdir+'/log.json','r') as f:
    data = f.readlines()

keys = json.loads(data[0]).keys()
d = {k:[] for k in keys}

if names is None:
    names = [k for k in keys if k not in ['epoch', 'batch', 'iteration']]

for i, line in enumerate(data):
    if not limits == None and (i < limits[0] or i > limits[1]):
        continue
    dat = json.loads(line)
    for k in keys:
        d[k].append(dat[k])

d = {k:np.array(d[k]) for k in keys}

In [ ]:
fstr = '%-20s %-6s %-6s %-6s %12s %12s'
print(fstr % ('name', 'len', 'nan', 'inf', 'min', 'max'))
for k,v in d.items():
    print(fstr % (
        k, 
        len(v),
        np.any(np.isnan(v)), 
        np.any(np.logical_not(np.isfinite(v))),
        round(np.min(v), 6),
        round(np.max(v), 6),
    ))

In [ ]:
plt.figure()
plt.plot(d['batch']+d['epoch']*max(d['batch'])-d['iteration'])
plt.show()
d['iteration'][-1]

In [ ]:
plt.figure()
k = 'fmeasure'
#k = 'recall'
#plt.plot(d['iteration'], d[k])
window_length = 2000
w = np.hanning(window_length)
wh = int(window_length/2)
x = d['iteration'][wh-1:-wh]
y = np.convolve(w/w.sum(), d[k], mode='valid')
plt.plot(x, y)
plt.ylim([0,0.5])
plt.grid()
plt.show()

In [ ]:
f = !df -h 
print(f[0], f[3], sep='\n')
f = !nvidia-smi
print('', f[5], f[8], sep='\n')